In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Vertex AI原始预测功能开始TensorFlow服务函数

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/get_started_with_raw_predict.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fprediction%2Fget_started_with_raw_predict.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/prediction/get_started_with_raw_predict.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/prediction/get_started_with_raw_predict.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub中查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用 `Vertex AI Raw Prediction` 将原始的 HTTP 内容直接发送到部署在 `Vertex AI Endpoint` 上的模型。

例如，预构建的 `Vertex AI` 部署容器的 HTTP 服务器不支持用于 TensorFlow 1.x 估计器的 HTTP 请求体。使用原始预测，可以通过 HTTP 服务器发送原始内容，这些内容会按原样呈现给模型输入--没有经过规范处理。

了解更多关于 [Raw Predict](https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.endpoints/rawPredict)。

### 目标

在本教程中，您将学习如何在 `Vertex AI Endpoint` 资源上使用 `Vertex AI Raw Prediction`。

本教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AI Raw Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 下载预训练的用于 TensorFlow 1.x 评估器的表格分类模型工件。
- 将 TensorFlow 评估器模型上传为 `Vertex AI Model` 资源。
- 创建一个 `Endpoint` 资源。
- 将 `Model` 资源部署到 `Endpoint` 资源。
- 对部署到 `Endpoint` 资源的 `Model` 资源实例进行在线原始预测。

### 数据集

本教程使用了一个预先训练的表格分类模型，数据存储在一个公共云存储桶中，该模型是在企鹅数据集（https://cloud.google.com/bigquery/public-data）上训练的。数据集的版本预测物种。

费用

本教程使用Google Cloud的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

开始吧

安装用于 Python 的 Vertex AI SDK 和其他必需的包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform 
! pip3 install tensorflow-hub

重新启动运行时（仅适用于Colab）

要使用新安装的软件包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核即将重新启动。在继续进行下一步之前，请等待它完成。⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并初始化用于Python的Vertex AI SDK

要开始使用Vertex AI，您必须具有现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储诸如数据集等中间产品。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶还不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 设置 Google Cloud 项目

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `DEPLOY_GPU/DEPLOY_NGPU` 来使用支持 GPU 的容器映像，并指定分配给虚拟机实例的 GPU 数量。例如，要使用一个支持 GPU 的容器映像，并且为每个虚拟机分配 4 个 Nvidia Tesla K80 GPU，您可以指定：

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定 `(None, None)` 来使用一个在 CPU 上运行的容器映像。

了解更多关于在您的地区支持的[硬件加速器](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置预构建的容器

设置用于预测的预构建Docker容器镜像。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TF = "2-5"

if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)


DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB内存。
     - `n1-highmem`: 每个vCPU 6.5GB内存
     - `n1-highcpu`: 每个vCPU 0.9GB内存
 - `vCPUs`: 数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

从公共云存储位置获取预训练模型

为了演示目的，本教程使用了一个预训练模型TensorFlow 1.x的估算器表格分类模型，然后将其上传到`Vertex AI Model`资源中。一旦您拥有了`Vertex AI Model`资源，该模型就可以部署到`Vertex AI Endpoint`资源中。

### 下载预训练模型

从公共云存储下载预训练的TensorFlow估算器模型工件，然后将模型工件上传到您自己的云存储存储桶中。

In [ ]:
MODEL_DIR = BUCKET_URI + "/model"

! gsutil cp -r gs://cloud-samples-data/vertex-ai/google-cloud-aiplatform-ci-artifacts/models/penguins/estimator/ {MODEL_DIR}

将TensorFlow估算器模型上传到`Vertex AI Model`资源

最后，将TFHub模型和serving功能的模型工件上传到`Vertex AI Model`资源。

*注意:*当您将模型工件上传到`Vertex AI Model`资源时，需要指定相应的部署容器镜像。

In [ ]:
model = aiplatform.Model.upload(
    display_name="example",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
)

print(model)

创建`Endpoint`资源

您可以使用`Endpoint.create()`方法创建一个`Endpoint`资源。至少，您需要指定端点的显示名称。可选的，您还可以指定项目和位置；否则，设置将继承您在使用`init()`方法初始化Vertex AI SDK时设置的值。

在这个例子中，以下参数被指定：

- `display_name`：`Endpoint`资源的可读名称。
- `project`：您的项目ID。
- `location`：您的位置。
- `labels`：(可选) 以键/值对形式提供的`Endpoint`的用户定义的元数据。

该方法返回一个`Endpoint`对象。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aiplatform.Endpoint.create(
    display_name="example",
    project=PROJECT_ID,
    location=LOCATION,
    labels={"your_key": "your_value"},
)

print(endpoint)

## 将 `Model` 资源部署到 `Endpoint` 资源。

您可以将一个或多个 `Vertex AI Model` 资源实例部署到同一个端点。每个被部署的 `Vertex AI Model` 资源将拥有自己的服务二进制容器。

在下面的示例中，您将 `Vertex AI Model` 资源部署到一个 `Vertex AI Endpoint` 资源。`Vertex AI Model` 资源已经定义了部署容器镜像。为了部署，您需要指定以下额外的配置设置：

- 机器类型。
- GPU 的类型和数量（如果有的话）。
- 静态、手动或自动扩展 VM 实例。

在这个示例中，您将以最少的指定参数部署模型，具体如下：

- `model`：`Model` 资源。
- `deployed_model_displayed_name`：部署模型实例的人类可读名称。
- `machine_type`：每个 VM 实例的机器类型。

由于资源预配的要求，这可能会花费几分钟的时间。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="example",
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

#### 创建预测实例

接下来，您可以使用一个合成示例准备一个预测请求。在这个示例中，模型格式是TensorFlow 1.x的estimator格式。这种模型格式采用了一个请求签名，并不被`Vertex AI`预构建的TensorFlow serving容器的HTTP服务器支持。

对于这种模型格式，您可以使用`raw_predict()`来直接传递一个符合模型服务接口的请求，请求格式如下：

    http_body -> {
        'signature_name' : serving_signature,
        'instances': [ {instance_1}, {instance_2}, ... ]
    }

    instance -> { 'feature_1': value_1, 'feature_2': value_2, ... }

    serving_signature -> "predict"

In [ ]:
import json

from google.api import httpbody_pb2
from google.cloud import aiplatform_v1

DATA = {
    "signature_name": "predict",
    "instances": [
        {
            "island": "DREAM",
            "culmen_length_mm": 36.6,
            "culmen_depth_mm": 18.4,
            "flipper_length_mm": 184.0,
            "body_mass_g": 3475.0,
            "sex": "FEMALE",
        }
    ],
}

http_body = httpbody_pb2.HttpBody(
    data=json.dumps(DATA).encode("utf-8"),
    content_type="application/json",
)

req = aiplatform_v1.RawPredictRequest(
    http_body=http_body, endpoint=endpoint.resource_name
)

做一个预测

最后，您可以使用Vertex AI预测服务发起预测请求。

In [ ]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(LOCATION)
client_options = {"api_endpoint": API_ENDPOINT}

pred_client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

response = pred_client.raw_predict(req)
print(response)

## 清理工作

要清理此项目中使用的所有 Google Cloud 资源，您可以[删除用于本教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False
delete_model = True
delete_endpoint = True

if delete_endpoint:
    try:
        endpoint.undeploy_all()
        endpoint.delete()
    except Exception as e:
        print(e)

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}